Importando dependências

In [ ]:
try:
    import openai
    import re
    import json

except ModuleNotFoundError:
    !pip install openai==0.28
    import openai
    import re
    import json

In [ ]:
!openai migrate
!pip install openai==0.28

Definição de chave e modelo

In [ ]:
openai.api_key = "SECRET-KEY"
MODELO = "gpt-3.5-turbo"

Prompt inicial

In [ ]:
prompt_base = f'''
Atue como um professor de língua portuguesa que deve avaliar e pontuar redações de acordo com 5 competências, que são:
Competência 1: Demonstrar domínio da modalidade escrita formal da língua portuguesa.
Competência 2: Compreender a proposta de redação e aplicar conceitos das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo em prosa.
Competência 3: Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.
Competência 4: Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação.
Competência 5: Elaborar proposta de intervenção para o problema abordado que respeite os direitos humanos.
Cada competência apenas pode receber uma nota (200, 160, 120, 80, 40 ou 0), dentre 6 classificações para cada competência que serão apresentadas posteriormente. Para isso, premissas serão adicionadas para guiar a avaliação. Entenda atentamente as premissas e avalie a redação, pontuando-a em apenas uma das 6 classificações.

Para avaliar a redação submetida na Competência 1, leve em consideração as premissas abaixo e classifique a pontuação para a Competência 1 de acordo com as Classificações abaixo.

Premissas:
Quanto aos desvios, você deve estar atento aos seguintes aspectos:
Convenções da escrita: acentuação, ortografia, uso de hífen, emprego de letras maiúsculas e minúsculas e separação silábica (translineação);
Gramaticais: regência verbal e nominal, concordância verbal e nominal, tempos e modos verbais, pontuação, paralelismo sintático, emprego de pronomes e crase;
Escolha de registro: adequação à modalidade escrita formal, isto é, ausência de uso de registro informal e/ou de marcas de oralidade;
Escolha vocabular: emprego de vocabulário preciso, o que significa que as palavras selecionadas são usadas em seu sentido correto e são apropriadas ao contexto em que aparecem.

Classificações:
Se o autor demonstra excelente domínio da modalidade escrita formal da língua portuguesa e de escolha de registro. Se desvios gramaticais ou de convenções da escrita serão aceitos somente como excepcionalidade e quando não caracterizam reincidência, atribua a nota 200 para a Competência 1.
Se o autor demonstra bom domínio da modalidade escrita formal da língua portuguesa e de escolha de registro, com poucos desvios gramaticais e de convenções da escrita, atribua a nota 160 para a Competência 1.
Se o autor demonstra domínio mediano da modalidade escrita formal da língua portuguesa e de escolha de registro, com alguns desvios gramaticais e de convenções da escrita, atribua a nota 120 para a Competência 1.
Se o autor demonstra domínio insuficiente da modalidade escrita formal da língua portuguesa, com muitos desvios gramaticais, de escolha de registro e de convenções de escrita, atribua a nota 80 para a Competência 1.
Se o autor demonstra domínio precário da modalidade escrita formal da língua portuguesa, de forma sistemática, com diversificados e frequentes desvios gramaticais, de escolha de registro e de convenções da escrita, atribua a nota 40 para a Competência 1.
Se o autor demonstra desconhecimento da modalidade escrita forma da língua portuguesa, atribua a nota 0 para a Competência 1.

Para avaliar a redação submetida na Competência 2, leve em consideração as premissas abaixo e classifique a pontuação para a Competência 2 de acordo com as Classificações abaixo.

Premissas:
O segundo aspecto a ser avaliado no seu texto é a compreensão da proposta de redação, composta por um tema específico a ser desenvolvido na forma de texto dissertativo-argumentativo — ou seja, a proposta exige que o autor escreva um texto dissertativo-argumentativo, que é um texto em que se demonstra, por meio de argumentação, a assertividade de uma ideia ou de um ponto de vista. É mais do que uma simples exposição de ideias; por isso, o autor deve evitar elaborar um texto de caráter apenas expositivo, devendo assumir claramente um ponto de vista. Além disso, é preciso que o ponto de vista que o autor irá defender esteja relacionado ao tema definido na proposta.
O tema constitui o núcleo das ideias sobre as quais o ponto de vista se organiza e é caracterizado por ser uma delimitação de um assunto mais abrangente. Por isso, o autor deve atender ao recorte temático definido para evitar tangenciá-lo (abordar parcialmente o tema) ou, ainda pior, desenvolver um tema distinto do determinado pela proposta. Outro aspecto avaliado na Competência 2 é a presença de repertório sociocultural, que se configura como uma informação, um fato, uma citação ou uma experiência vivida que, de alguma forma, contribui como argumento para a discussão proposta.

Classificações:
Se o autor desenvolve o tema por meio de argumentação consistente, a partir de um repertório sociocultural produtivo, e apresenta excelente domínio do texto dissertativo-argumentativo, atribua a nota 200 para a Competência 2.
Se o autor desenvolve o tema por meio de argumentação consistente e apresenta bom domínio do texto dissertativo-argumentativo, com proposição, argumentação e conclusão, atribua a nota 160 para a Competência 2.
Se o autor desenvolve o tema recorrendo à cópia de trechos dos textos motivadores ou apresenta domínio insuficiente do texto dissertativo-argumentativo, não atendendo à estrutura com proposição, argumentação e conclusão, , atribua a nota 120 para a Competência 2.
Se o autor desenvolve o tema por meio de argumentação previsível e apresenta domínio mediano do texto dissertativo-argumentativo, com proposição, argumentação e conclusão, atribua a nota 80 para a Competência 2.
Se o autor apresenta o assunto, tangenciando o tema, ou demonstra domínio precário do texto dissertativo-argumentativo, com traços constantes de outros tipos textuais, atribua a nota 40 para a Competência 2.
Se o autor foge ao tema/não atendimento à estrutura dissertativo-argumentativa. Nestes casos, a redação recebe nota zero e é anulada, atribua a nota 0 para a Competência 2.

Para avaliar a redação submetida na Competência 3, leve em consideração as premissas abaixo e classifique a pontuação para a Competência 3 de acordo com as Classificações abaixo.

Premissas:
Na organização do texto dissertativo-argumentativo, o autor deve procurar atender às seguintes exigências:
- apresentação clara do ponto de vista e seleção dos argumentos que o sustentam;
- encadeamento das ideias, de modo que cada parágrafo apresente informações coerentes com o que foi apresentado anteriormente, sem repetições desnecessárias ou saltos temáticos (mudanças abruptas sobre o que está sendo discutido);
- desenvolvimento dessas ideias por meio da explicitação, explicação ou exemplificação de informações, fatos e opiniões, de modo a justificar, para o leitor, o ponto de vista escolhido.

Classificações:
Se o autor apresenta informações, fatos e opiniões relacionados ao tema proposto, de forma consistente e organizada, configurando autoria, em defesa de um ponto de vista, atribua a nota 200 para a Competência 3.
Se o autor apresenta informações, fatos e opiniões relacionados ao tema, de forma organizada, com indícios de autoria, em defesa de um ponto de vista, atribua a nota 160 para a Competência 3.
Se o autor apresenta informações, fatos e opiniões relacionados ao tema, limitados aos argumentos dos textos motivadores e pouco organizados, em defesa de um ponto de vista, atribua a nota 120 para a Competência 3.
Se o autor apresenta informações, fatos e opiniões relacionados ao tema, mas desorganizados ou contraditórios e limitados aos argumentos dos textos motivadores, em defesa de um ponto de vista, atribua a nota 80 para a Competência 3.
Se o autor apresenta informações, fatos e opiniões pouco relacionados ao tema ou incoerentes e sem defesa de um ponto de vista, atribua a nota 40 para a Competência 3.
Se o autor apresenta informações, fatos e opiniões não relacionados ao tema e sem defesa de um ponto de vista, atribua a nota 0 para a Competência 3.

Para avaliar a redação submetida na Competência 4, leve em consideração as premissas abaixo e classifique a pontuação para a Competência 4 de acordo com as Classificações abaixo.

Premissas:
Para garantir a coesão textual, devem ser observados na redação determinados princípios em diferentes níveis:
- estruturação dos parágrafos – um parágrafo é uma unidade textual formada por uma ideia principal à qual se ligam ideias secundárias. No texto dissertativo-argumentativo, os parágrafos podem ser desenvolvidos por comparação, por causa-consequência, por exemplificação, por detalhamento, entre outras possibilidades. Deve haver articulação explícita entre um parágrafo e outro;
- estruturação dos períodos – pela própria especificidade do texto dissertativo-argumentativo, os períodos do texto são, normalmente, estruturados de modo complexo, formados por duas ou mais orações, para que se possam expressar as ideias de causa/consequência, contradição, temporalidade, comparação, conclusão, entre outras;
- referenciação – pessoas, coisas, lugares e fatos são apresentados e, depois, retomados, à medida que o texto vai progredindo. Esse processo pode ser realizado mediante o uso de pronomes, advérbios, artigos, sinônimos, antônimos, hipônimos, hiperônimos, além de expressões resumitivas, metafóricas ou metadiscursivas.
O autor deve usar estratégias de coesão para se referir a elementos que já apareceram no texto:
- substituição de termos ou expressões por pronomes pessoais, possessivos e demonstrativos, advérbios que indicam localização, artigos;
- substituição de termos ou expressões por sinônimos, hipônimos, hiperônimos ou expressões resumitivas;
- substituição de verbos, substantivos, períodos ou fragmentos do texto por conectivos ou expressões que retomem o que foi dito;
- elipse ou omissão de elementos que já tenham sido citados ou que sejam facilmente identificáveis.
O autor deveria evitar:
- ausência de articulação entre orações, frases e parágrafos;
- ausência de paragrafação (texto elaborado em um único parágrafo);
- emprego de conector (preposição, conjunção, pronome relativo, alguns advérbios e locuções adverbiais) que não estabeleça relação lógica entre dois trechos do texto e prejudique a compreensão da mensagem;
- repetição ou substituição inadequada de palavras sem se valer dos recursos oferecidos pela língua (pronome, advérbio, artigo, sinônimo).

Classificações:

Se o autor articula bem as partes do texto e apresenta repertório diversificado de recursos coesivos, atribua a nota 200 para a Competência 4.
Se o autor articula as partes do texto, com poucas inadequações, e apresenta repertório diversificado de recursos coesivos, atribua a nota 160 para a Competência 4.
Se o autor articula as partes do texto, de forma mediana, com inadequações, e apresenta repertório pouco diversificado de recursos coesivos, atribua a nota 120 para a Competência 4.
Se o autor articula as partes do texto, de forma insuficiente, com muitas inadequações, e apresenta repertório limitado de recursos coesivos, atribua a nota 80 para a Competência 4.
Se o autor articula as partes do texto de forma precária, atribua a nota 40 para a Competência 4.
Se o autor não articula as informações, atribua a nota 0 para a Competência 4.

Para avaliar a redação submetida na Competência 5, leve em consideração as premissas abaixo e classifique a pontuação para a Competência 5 de acordo com as Classificações abaixo.

Premissas:
Ao elaborar sua proposta de intervenção, o autor deve procurar responder às seguintes perguntas:
1. O que é possível apresentar como solução para o problema?
2. Quem deve executá-la?
3. Como viabilizar essa solução?
4. Qual efeito ela pode alcançar?
5. Que outra informação pode ser acrescentada para detalhar a proposta?
Para a avaliação das redações, são considerados os seguintes princípios norteadores dos direitos humanos, pautados no artigo 3o da Resolução no 1, de 30 de maio de 2012, a qual estabelece as Diretrizes Nacionais para a Educação em Direitos Humanos:
- Dignidade humana.
- Igualdade de direitos.
- Reconhecimento e valorização das diferenças e diversidades.
- Laicidade do Estado.
- Democracia na educação.
- Transversalidade, vivência e globalidade.
- Sustentabilidade socioambiental.

Classificações:

Se o autor elabora muito bem proposta de intervenção, detalhada, relacionada ao tema e articulada à discussão desenvolvida no texto, atribua a nota 200 para a Competência 5.
Se o autor elabora bem proposta de intervenção relacionada ao tema e articulada à discussão desenvolvida no texto, atribua a nota 160 para a Competência 5.
Se o autor elabora, de forma mediana, proposta de intervenção relacionada ao tema e articulada à discussão desenvolvida no texto, atribua a nota 120 para a Competência 5.
Se o autor elabora, de forma insuficiente, proposta de intervenção relacionada ao tema, ou não articulada com a discussão desenvolvida no texto, atribua a nota 80 para a Competência 5.
Se o autor apresenta proposta de intervenção vaga, precária ou relacionada apenas ao assunto, atribua a nota 40 para a Competência 5.
Se o autor não apresenta proposta de intervenção ou apresenta proposta não relacionada ao tema ou ao assunto, atribua a nota 0 para a Competência 5.

Apresente e discuta cada uma das 5 competências, sempre citando trechos da redação original que fazem com que a redação perca nota, para cada uma das 5 competências. Portanto, sempre mostre os trechos da redação original para exemplificar a crítica. No final, faça uma lista de críticas positivas sobre a redação, de acordo com as competências, premissas e classificações. Não mostre pontuações, ne, pontuações sugeridas. No final, não faça comentários adicionais sobre as críticas.
'''

prompt_chatgpt_base = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": prompt_base},
        ])

Inserção do tema

In [ ]:
prompt_tema = input("Digite o tema da redação: ")



prompt_chatgpt_tema = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": prompt_tema},
        ])

Inserção da redação

In [ ]:
prompt_redacao = input("Digite a redação a ser avaliada: ")

prompt_chatgpt_redacao = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": prompt_redacao},
        ])


Avaliação

In [ ]:
prompt_avaliacao = f'''
Apresente e discuta cada uma das 5 competências, sempre citando trechos da redação original que fazem com que a redação perca nota, para cada uma das 5 competências. Portanto, sempre mostre os trechos da redação original para exemplificar a crítica. No final, faça uma lista de críticas positivas sobre a redação, de acordo com as competências, premissas e classificações. Não mostre pontuações. No final, não faça juízo de valor comparando a quantidade de críticas negativas e positivas. Para realizar as avaliações e críticas, embora a revisão e a adição de detalhes mais específicos e aprofundados pudessem elevar a redação para o nível mais alto, não se exige excelência em todos os aspectos para a nota 200 em cada Competência, ou mesmo a superação de expectativas em cada uma das competências, para que obtenha um escore superior de 1000. Isso significa que críticas que não tem grande impacto na avaliação final, não devem aparecer.
'''

prompt_chatgpt_avaliacao = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": prompt_avaliacao},
        ])

Mostrar resultado

In [ ]:
resposta_final = "{}".format(prompt_chatgpt_avaliacao.choices[0]["message"]["content"])
print(resposta_final)